In [1]:
import itertools
import pandas as pd
import requests
import pickle

### Scaper
https://www.msc.com/en/search-a-schedule

In [2]:
with open('../pickles/msc_country_port_codes.pickle', 'rb') as handle:
    country_port_codes = pickle.load(handle)

In [3]:
print(country_port_codes["Countries"][:2])
print(" - "*20)
print(country_port_codes["Ports"][:2])

[{'CountryName': 'AFGHANISTAN', 'CountryIsoCode': 'AF', 'CountryId': '29'}, {'CountryName': 'ALBANIA', 'CountryIsoCode': 'AL', 'CountryId': '41'}]
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
[{'PortId': 896, 'LocationName': 'AALBORG', 'LocationCode': 'DKAAL', 'CountryName': 'DENMARK', 'CountryIsoCode': 'DK', 'LocationLatitude': 57.047036, 'LocationLongitude': 10.055766}, {'PortId': 424, 'LocationName': 'AARHUS', 'LocationCode': 'DKAAR', 'CountryName': 'DENMARK', 'CountryIsoCode': 'DK', 'LocationLatitude': 56.153475, 'LocationLongitude': 10.240799}]


In [4]:
# Create list of origin and destination ports
o_ports = ['BRFOR', 'BRSSZ', 'BRPEC', 'BRSSA', 'BRPNG', 'BRRIO', 'BRRIG', 'BRITJ', 'BRNAT', 'COCTG', 'COSMR', 'COTRB', 'SRPBM', 'CWWIL', 'UYMVD', 'ARBUE', 'CLCNL', 'CLSAI', 'CLVAP', 'PEPAI', 'PECLL', 'ECGYE', 'ECPBO', 'ECPSJ', 'VNVUT', 'VNCMT']
d_ports = ['NLRTM', 'BEANR', 'NLVLI', 'BEZEE', 'NLAMS']

# Make list with all combinations
od_ports = list(itertools.product(o_ports, d_ports))
print(f"{len(od_ports)} combinations of ports")

# Define the base URL to search
base_url = "https://www.msc.com/en/search-a-schedule"

130 combinations of ports


In [5]:
# Use today's date, by default
from datetime import date
today = date.today()
print(today)

2022-10-31


In [6]:
o_port = "475"
d_port = "208"
direct = True
date = today

url = "https://www.msc.com/api/feature/tools/SearchSailingRoutes"

payload = {
    "FromDate": str(today),
    "fromPortId": o_port,
    "toPortId": d_port,
    "isDirectRouteOnly": direct,
    "language": "en"
}
headers = {
    "authority": "www.msc.com",
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "x-requested-with": "XMLHttpRequest"
}

response = requests.request("POST", url, json=payload, headers=headers)

rdict = response.json()
succes = rdict["IsSuccess"]
print(succes)
data = rdict["Data"][0]
print(len(data["Routes"]))

True
7


In [7]:
data["Routes"][0]

{'CO2FootPrint': '0.7926 Tons',
 'CutOffs': {'ContainerYardCutOffDate': 'Tue 1st Nov 2022 17:00',
  'ReeferCutOffDate': 'Tue 1st Nov 2022 17:00',
  'DangerousCargoCutOffDate': 'Thu 27th Oct 2022 12:00',
  'ShippingInstructionsCutOffDate': 'Fri 28th Oct 2022 12:00',
  'VerifiedGrossMassCutOffDate': 'Tue 1st Nov 2022 15:00'},
 'EstimatedDepartureDateFormatted': 'Fri 4th Nov 2022',
 'EstimatedDepartureDate': '2022-11-04T22:00:00',
 'EstimatedArrivalDateFormatted': 'Wed 23rd Nov 2022',
 'EstimatedArrivalDate': '2022-11-23T16:00:00',
 'VesselName': 'MSC SOFIA CELESTE',
 'DepartureVoyageNo': 'NA243R',
 'TotalTransitTime': '19 days',
 'TotalTransitTimeHours': 456,
 'RouteScheduleLegDetails': [{'LegSequence': 1,
   'SingleLegOnly': True,
   'LastLeg': True,
   'DepartureVoyageNo': 'NA243R',
   'DeparturePortUNCode': 'BRSSZ',
   'DeparturePortId': 475,
   'DeparturePortName': 'SANTOS',
   'DepartureEquipmentHandlingFacilityName': 'BRASIL TERMINAL PORTUARIO S/A',
   'EstimatedDepartureTime': '20

In [8]:
pd.DataFrame(data["Routes"])

,CO2FootPrint,CutOffs,EstimatedDepartureDateFormatted,EstimatedDepartureDate,EstimatedArrivalDateFormatted,EstimatedArrivalDate,VesselName,DepartureVoyageNo,TotalTransitTime,TotalTransitTimeHours,RouteScheduleLegDetails
0,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 1st Nov 2022 ...,Fri 4th Nov 2022,2022-11-04T22:00:00,Wed 23rd Nov 2022,2022-11-23T16:00:00,MSC SOFIA CELESTE,NA243R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
1,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 8th Nov 2022 ...,Sat 12th Nov 2022,2022-11-12T21:00:00,Wed 30th Nov 2022,2022-11-30T16:00:00,CZECH,NA244R,18 days,432,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
2,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 15th Nov 2022...,Fri 18th Nov 2022,2022-11-18T09:00:00,Wed 7th Dec 2022,2022-12-07T16:00:00,MSC ATHENS,NA245R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
3,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 22nd Nov 2022...,Fri 25th Nov 2022,2022-11-25T09:00:00,Wed 14th Dec 2022,2022-12-14T16:00:00,MSC PALAK,NA246R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
4,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 29th Nov 2022...,Fri 2nd Dec 2022,2022-12-02T09:00:00,Wed 21st Dec 2022,2022-12-21T16:00:00,HUNGARY,NA247R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
5,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 6th Dec 2022 ...,Fri 9th Dec 2022,2022-12-09T09:00:00,Wed 28th Dec 2022,2022-12-28T16:00:00,MSC CATERINA,NA248R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
6,0.7926 Tons,{'ContainerYardCutOffDate': 'Tue 20th Dec 2022...,Fri 23rd Dec 2022,2022-12-23T09:00:00,Wed 11th Jan 2023,2023-01-11T16:00:00,UASC ZAMZAM,NA250R,19 days,456,"[{'LegSequence': 1, 'SingleLegOnly': True, 'La..."
